In [234]:
# Import required modules


import psycopg2
import pandas as pd
import numpy as np
import datetime

# If you don't have the psycopg2 package, connect to Shell VPN and install this in a cmd.
# conda install -c anaconda psycopg2


In [235]:
# User input

## redshift username and password
username = 
password = 

## please manualy adjust outage data range in formate 2020-01-01 and sales data range in sql query


In [236]:
# This code first requires SQL query to get data from the database. 


# Edit your username and password here

con=psycopg2.connect(dbname= 'feftorrance', host='torrance-redshift-redshiftcluster-1s02pngglgmmy.cybkoauambcw.us-east-1.redshift.amazonaws.com',
port= '5439', user= username, password= password)
cur = con.cursor()


#sql query outage data
sql_outage = '''SELECT
 sitecontrollername as pos_name,
 case
    when position('Mission' in sitecontrollername) > 0 then 'Mission Street'
    when position('Citrus Heights' in sitecontrollername) > 0 then 'Citrus Heights'
    when position('Fair Oaks' in sitecontrollername) > 0 then 'Fair Oaks'
    when position('Third Street' in sitecontrollername) > 0 then 'Third Street'
    else 'zzz'
  end as station_name,
  eventdate,
  eventid
FROM comdata_pos_vendor.site_event
WHERE eventid = 652 and eventdate between '2020-01-01' and '2020-12-31' and station_name != 'zzz' 
      or eventid = 653 and eventdate between '2020-01-01' and '2020-12-31' and station_name != 'zzz'
ORDER BY pos_name, eventdate, eventid
;'''
outage = pd.read_sql_query(sql_outage, con)


#sql query sales data
sql_sales = '''select
  case
    when position('Mission' in pos_name) > 0 then 'Mission Street'
    when position('Citrus' in pos_name) > 0 then 'Citrus Heights'
    when position('Fair' in pos_name) > 0 then 'Fair Oaks'
    when position('Third' in pos_name) > 0 then 'Third Street'
    else 'zzz'
  end as station_name,
  extract(year from transaction_date) || '-' || extract(month from transaction_date) ||'-' || extract(day from transaction_date) as sales_date,
  sum(quantity_kg) as daily_sales
from comdata_pos_study.fueling_api_table_02
where station_name != 'zzz' and extract(year from transaction_date) = 2020
group by station_name, sales_date
order by station_name, sales_date
;'''
sales = pd.read_sql_query(sql_sales, con)


# close sql query
cur.close()
con.close()

outage


,pos_name,station_name,eventdate,eventid
0,Citrus Heights A,Citrus Heights,2020-02-10 11:32:28.043,653
1,Citrus Heights A,Citrus Heights,2020-02-10 18:38:54.867,652
2,Citrus Heights A,Citrus Heights,2020-02-10 18:39:49.177,653
3,Citrus Heights A,Citrus Heights,2020-02-12 06:05:08.610,652
4,Citrus Heights A,Citrus Heights,2020-02-12 06:15:04.123,653
...,...,...,...,...
6815,Third Street B,Third Street,2020-12-29 09:51:45.193,652
6816,Third Street B,Third Street,2020-12-29 10:23:22.283,652
6817,Third Street B,Third Street,2020-12-29 17:10:43.170,652
6818,Third Street B,Third Street,2020-12-29 20:26:22.933,652


In [237]:
# Select the beginning and the end of the outage event

## add new column
new_cols = ['select 1']
outage = outage.reindex(outage.columns.union(new_cols), axis=1)

if outage.iloc[0,1]==652:
    outage.iloc[0,3]=1
else: outage.iloc[0,3]=0


## for loop to assign value 1 to the beginning and end of outage event and 0 to the rest

length = len(outage)

for x in range(1, length):
    if outage.iloc[x,1]==653:
        if outage.iloc[x-1,1] == 652 and outage.iloc[x,2] == outage.iloc[x-1,2]:
            outage.iloc[x,3]=1
        else: outage.iloc[x,3]=0
    elif outage.iloc[x-1,1] == 653 or outage.iloc[x,2] != outage.iloc[x-1,2]:
            outage.iloc[x,3]=1
    else: outage.iloc[x,3]=0

## select when value = 1
outage_s = outage.loc[outage['select 1']==1]
outage_s = outage_s.reset_index(drop=True)

outage_s



,eventdate,eventid,pos_name,select 1,station_name
0,2020-02-10 18:38:54.867,652,Citrus Heights A,1.0,Citrus Heights
1,2020-02-10 18:39:49.177,653,Citrus Heights A,1.0,Citrus Heights
2,2020-02-12 06:05:08.610,652,Citrus Heights A,1.0,Citrus Heights
3,2020-02-12 06:15:04.123,653,Citrus Heights A,1.0,Citrus Heights
4,2020-02-14 13:26:15.913,652,Citrus Heights A,1.0,Citrus Heights
...,...,...,...,...,...
6640,2020-12-24 06:10:34.380,652,Third Street B,1.0,Third Street
6641,2020-12-24 07:24:52.227,653,Third Street B,1.0,Third Street
6642,2020-12-24 08:43:42.367,652,Third Street B,1.0,Third Street
6643,2020-12-24 11:26:15.277,653,Third Street B,1.0,Third Street


In [238]:
## for loop to eliminate some 652 records at the end of each POS

outage_s.iloc[0,3]= 1
length = len(outage_s)

for x in range(1, length-1):
    if outage_s.iloc[x,1]==653:
        if outage_s.iloc[x,1] - 1 == outage_s.iloc[x-1,1] :
            outage_s.iloc[x,3]=1
        else: outage_s.iloc[x,3]=0
    elif outage_s.iloc[x,1] + 1 == outage_s.iloc[x+1,1]:
            outage_s.iloc[x,3]=1
    else: outage_s.iloc[x,3]=0

if outage_s.iloc[-1,1]==652:
    outage_s.drop(outage_s.index[-1],inplace=True)
else: None


outage_s = outage_s.loc[outage_s['select 1']==1].drop(['select 1'],axis = 1).reset_index(drop=True)

outage_s



,eventdate,eventid,pos_name,station_name
0,2020-02-10 18:38:54.867,652,Citrus Heights A,Citrus Heights
1,2020-02-10 18:39:49.177,653,Citrus Heights A,Citrus Heights
2,2020-02-12 06:05:08.610,652,Citrus Heights A,Citrus Heights
3,2020-02-12 06:15:04.123,653,Citrus Heights A,Citrus Heights
4,2020-02-14 13:26:15.913,652,Citrus Heights A,Citrus Heights
...,...,...,...,...
6637,2020-12-24 06:07:16.597,653,Third Street B,Third Street
6638,2020-12-24 06:10:34.380,652,Third Street B,Third Street
6639,2020-12-24 07:24:52.227,653,Third Street B,Third Street
6640,2020-12-24 08:43:42.367,652,Third Street B,Third Street


In [239]:
# select records when time elapsed between down time start and down timme end was greater than 1 hour

## create two separate datasets, one for 652 and one for 653. Concat them together.
outage_s_652 = outage_s.loc[outage_s['eventid']==652].reset_index(drop=True).rename(columns={'eventdate': 'eventdate_652'})
outage_s_653 = outage_s.loc[outage_s['eventid']==653].reset_index(drop=True).rename(columns={'eventdate': 'eventdate_653'})
outage_c = pd.concat([outage_s_652,outage_s_653],axis=1)


## calculate hour of day and time elapsed between 652 and 653.
outage_c['timediff_hour'] = (outage_c.eventdate_653-outage_c.eventdate_652).astype('timedelta64[h]')
outage_c['timediff_day'] = (outage_c['timediff_hour']/24).apply(np.floor).astype('int')
outage_c['just_date']= outage_c['eventdate_652'].dt.date

## selecting time elapsed >= 1
outage_c = outage_c.loc[outage_c['timediff_hour']>0]
outage_c= outage_c.reset_index(drop=True)

## for time difference < 12 hours, only select when beginning hour is between 10 am and 7 pm

pd.options.mode.chained_assignment = None 
outage_c_sel2 = outage_c.loc[outage_c['timediff_hour']<=12]
outage_c_sel2['hour']= outage_c_sel2['eventdate_652'].dt.hour
outage_c_sel2['select 1'] = outage_c_sel2['eventdate_652'].dt.hour


for x in range (0,len(outage_c_sel2)):
    if outage_c_sel2.iloc[x,11] > 9 and outage_c_sel2.iloc[x,11] < 21:
        outage_c_sel2.iloc[x,12] = 1
    else: outage_c_sel2.iloc[x,12] = 0
    
outage_c_sel2 = outage_c_sel2.loc[outage_c_sel2['select 1']==1].drop(['select 1','hour'],axis = 1).reset_index(drop=True)

## append subset of data back

outage_c_sel1 = outage_c.loc[outage_c['timediff_hour']>12]
outage_c = pd.concat([outage_c_sel1,outage_c_sel2],axis=0).reset_index(drop=True)


outage_c


,eventdate_652,eventid,pos_name,station_name,eventdate_653,eventid,pos_name,station_name,timediff_hour,timediff_day,just_date
0,2020-02-17 07:46:41.383,652,Citrus Heights A,Citrus Heights,2020-02-17 21:01:54.867,653,Citrus Heights A,Citrus Heights,13.0,0,2020-02-17
1,2020-04-23 16:24:58.017,652,Citrus Heights A,Citrus Heights,2020-04-24 11:45:39.353,653,Citrus Heights A,Citrus Heights,19.0,0,2020-04-23
2,2020-04-25 11:12:06.027,652,Citrus Heights A,Citrus Heights,2020-04-27 08:23:45.490,653,Citrus Heights A,Citrus Heights,45.0,1,2020-04-25
3,2020-04-29 19:36:26.997,652,Citrus Heights A,Citrus Heights,2020-04-30 12:27:40.467,653,Citrus Heights A,Citrus Heights,16.0,0,2020-04-29
4,2020-04-30 23:55:34.657,652,Citrus Heights A,Citrus Heights,2020-05-04 08:44:22.157,653,Citrus Heights A,Citrus Heights,80.0,3,2020-04-30
...,...,...,...,...,...,...,...,...,...,...,...
545,2020-08-22 12:27:47.747,652,Third Street B,Third Street,2020-08-22 13:58:22.123,653,Third Street B,Third Street,1.0,0,2020-08-22
546,2020-08-23 16:26:34.497,652,Third Street B,Third Street,2020-08-23 17:33:24.213,653,Third Street B,Third Street,1.0,0,2020-08-23
547,2020-08-24 10:35:48.807,652,Third Street B,Third Street,2020-08-24 13:05:59.303,653,Third Street B,Third Street,2.0,0,2020-08-24
548,2020-09-06 18:49:06.370,652,Third Street B,Third Street,2020-09-06 22:03:39.227,653,Third Street B,Third Street,3.0,0,2020-09-06


In [240]:
# creating a outage date table with all dates between start date and end date of the down time event

outage_sche = pd.DataFrame(columns=['date','pos_name','station_name'])

tot = len(outage_c) + outage_c['timediff_day'].sum()
length = len(outage_c)
x = 0
y = 0

while (x < tot):
    while (y < length):
        a = 0
        z = outage_c.iloc[y,9]
        if z == 0:
            outage_sche.loc[x] = [outage_c.iloc[y,10],outage_c.iloc[y,2],outage_c.iloc[y,3]]
            x = x + 1
        else: 
            while (int(a) <= int(z)):
                outage_sche.loc[x] = [outage_c.iloc[y,10]+datetime.timedelta(days=a),outage_c.iloc[y,2],outage_c.iloc[y,3]]
                a = a + 1
                x = x + 1
        y = y + 1        

outage_sche
                

,date,pos_name,station_name
0,2020-02-17,Citrus Heights A,Citrus Heights
1,2020-04-23,Citrus Heights A,Citrus Heights
2,2020-04-25,Citrus Heights A,Citrus Heights
3,2020-04-26,Citrus Heights A,Citrus Heights
4,2020-04-29,Citrus Heights A,Citrus Heights
...,...,...,...
1211,2020-08-22,Third Street B,Third Street
1212,2020-08-23,Third Street B,Third Street
1213,2020-08-24,Third Street B,Third Street
1214,2020-09-06,Third Street B,Third Street


In [241]:
# create outage data at station level

## drop duplipcate rows
outage_sche.drop_duplicates(inplace=True)
outage_sche= outage_sche.reset_index(drop=True)

## group based on POS name A or B
outage_sche['group'] = outage_sche['pos_name'].str.strip().str[-1]
outage_sche_A = outage_sche.loc[outage_sche['group']=='A']
outage_sche_B = outage_sche.loc[outage_sche['group']=='B']

## concat two dataframes together to get outage list at station level
outage_sche_c = pd.merge(outage_sche_A,outage_sche_B,how='right',left_on=['date','station_name'], right_on = ['date','station_name'])
outage_station = outage_sche_c.dropna().drop(['pos_name_x', 'group_x','pos_name_y','group_y'], axis = 1) 
outage_station['outage']=1

outage_station


,date,station_name,outage
0,2020-04-25,Citrus Heights,1
1,2020-04-26,Citrus Heights,1
2,2020-05-03,Citrus Heights,1
3,2020-05-04,Citrus Heights,1
4,2020-05-05,Citrus Heights,1
...,...,...,...
295,2020-11-09,Mission Street,1
296,2020-04-24,Third Street,1
297,2020-06-12,Third Street,1
298,2020-08-01,Third Street,1


In [242]:
# Concat station outage data with sales data

## adjust data object to datetime
sales['sales_date']=sales['sales_date'].astype('datetime64[ns]')
outage_station['date']=outage_station['date'].astype('datetime64[ns]')

## calculate day of week and month from date
sales['month'] = pd.DatetimeIndex(sales['sales_date']).month
sales['dow']=pd.DatetimeIndex(sales['sales_date']).dayofweek

## concat two dataframes
sales_c = pd.merge(sales,outage_station,how='left',left_on=['sales_date','station_name'], right_on = ['date','station_name']).drop(['date'], axis = 1).fillna(0)

sales_c


,station_name,sales_date,daily_sales,month,dow,outage
0,Citrus Heights,2020-01-10,46.382,1,4,0.0
1,Citrus Heights,2020-01-11,37.095,1,5,0.0
2,Citrus Heights,2020-01-12,46.996,1,6,0.0
3,Citrus Heights,2020-01-13,40.804,1,0,0.0
4,Citrus Heights,2020-01-14,59.614,1,1,0.0
...,...,...,...,...,...,...
1379,Third Street,2020-09-05,25.290,9,5,0.0
1380,Third Street,2020-09-06,21.385,9,6,0.0
1381,Third Street,2020-09-07,19.489,9,0,0.0
1382,Third Street,2020-09-08,25.930,9,1,0.0


In [244]:
# calculate average at day of week by month by station

## subsetting data
sales_avg = sales_c.loc[sales_c['outage']==0].reset_index(drop=True)

## calculate mean by station name, month, dow
sales_avg_s= sales_avg.groupby(['station_name','month','dow'])['daily_sales'].mean().reset_index()
sales_avg_s['month_to_use']=sales_avg_s['month']+1
sales_avg_s = sales_avg_s.rename(columns={'daily_sales': 'avg'})

# match the average back to sales data
sales_a = pd.merge(sales_c,sales_avg_s,how='left',left_on=['station_name','month','dow'], right_on = ['station_name','month_to_use','dow']).drop(['month_x', 'dow','month_y','month_to_use'], axis = 1).fillna(0)

sales_a



,station_name,sales_date,daily_sales,outage,avg
0,Citrus Heights,2020-01-10,46.382,0.0,0.00000
1,Citrus Heights,2020-01-11,37.095,0.0,0.00000
2,Citrus Heights,2020-01-12,46.996,0.0,0.00000
3,Citrus Heights,2020-01-13,40.804,0.0,0.00000
4,Citrus Heights,2020-01-14,59.614,0.0,0.00000
...,...,...,...,...,...
1379,Third Street,2020-09-05,25.290,0.0,29.78850
1380,Third Street,2020-09-06,21.385,0.0,25.73875
1381,Third Street,2020-09-07,19.489,0.0,30.33160
1382,Third Street,2020-09-08,25.930,0.0,25.60550


In [247]:
# Calculate the loss revenue

length = len(sales_a)
new_cols=['adjusted_sales']
sales_a = sales_a.reindex(sales_a.columns.union(new_cols), axis=1)

for x in range (0, length):
    if sales_a.iloc[x,3] == 1: 
        if sales_a.iloc[x,2] < sales_a.iloc[x,1]:
            sales_a.iloc[x,0] = sales_a.iloc[x,1]
        else: sales_a.iloc[x,0] = sales_a.iloc[x,2]
    else: sales_a.iloc[x,0] = sales_a.iloc[x,2]

sales_f = sales_a.drop(['avg','outage'],axis=1)


result = sales_f.groupby('station_name').sum().reset_index()

result['lost_revenue($)'] = (result['adjusted_sales'] - result['daily_sales'])*16.45

print(result.round(0))



     station_name  adjusted_sales  daily_sales  lost_revenue($)
0  Citrus Heights         19842.0      17348.0          41025.0
1       Fair Oaks         13699.0      11694.0          32992.0
2  Mission Street          6415.0       5926.0           8040.0
3    Third Street          8330.0       7523.0          13282.0


In [64]:
#sales_f.to_excel(r"C:\Users\Patrick.Chen\Desktop\adjusted_sales_data.xlsx")